In [72]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import random
import re
import nltk

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')


ModuleNotFoundError: No module named 'nltk'

In [2]:
train_data = pd.read_csv('~/Documents/Datos/DataSets/TP2/train_featured.csv')
test_data = pd.read_csv('~/Documents/Datos/DataSets/TP2/test_featured.csv')
train_data.head()

,id_original,keyword_original,location_original,text_original,target_label,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,links_count,clean_text,text_length,mean_word_length,vowels_count,short_words_count,stopwords_count,words_count
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,1,#earthquake,NaN,1,0,0,0,deeds reason earthquake may allah forgive us,68,4.31,25,7,6,13
1,4,NaN,NaN,forest fire near la ronge sask canada,1,1,NaN,NaN,0,0,0,0,forest fire near la ronge sask canada,37,4.43,13,1,0,7
2,5,NaN,NaN,all residents asked to shelter in place are be...,1,3,NaN,NaN,0,0,0,0,residents asked shelter place notified officer...,130,4.95,45,9,11,22
3,6,NaN,NaN,people receive wildfires evacuation orders in ...,1,2,#wildfires,NaN,1,0,5,0,people receive wildfires evacuation orders cal...,56,7.14,24,1,1,7
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,2,#alaska #wildfires,NaN,2,0,0,0,got sent photo ruby alaska smoke wildfires pou...,85,4.38,25,3,7,16


In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [4]:
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import time

from gensim.models.word2vec import Word2Vec
from glob import glob

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Reshape, Flatten, Dropout, Concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

In [7]:
target = train_data['target_label'].values.tolist()
data = train_data['clean_text'].values.tolist()

In [8]:
x_train, x_test, y_train, y_test = \
    train_test_split(data,target, test_size=0.2)  # random split

In [9]:
tokenizer = Tokenizer()             #Tokenizer(num_words=5000) => 5000 words of the highest frequency
tokenizer.fit_on_texts(data)
tokenizer

In [10]:
print("len(tokenizer) :",len(list(tokenizer.word_index)))

len(tokenizer) : 14368


In [11]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)


In [12]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)


In [13]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [14]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

16

In [15]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.97425456456062

In [16]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)


In [17]:
#Zero is added before the values given in the padding operation.

print("x_train_tokens :",x_train_tokens[0])
print("x_train_pad :",x_train_pad[0])

x_train_tokens : [549, 677, 244, 1, 63, 1904, 1311, 1437, 160, 549, 677, 244, 1, 63, 1904]
x_train_pad : [   0  549  677  244    1   63 1904 1311 1437  160  549  677  244    1
   63 1904]


In [18]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)


In [19]:
print("x_train_pad.shape :",x_train_pad.shape)
print("x_train_pad.shape :",x_test_pad.shape)

x_train_pad.shape : (6090, 16)
x_train_pad.shape : (1523, 16)


In [20]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [21]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [22]:
x_train[800]


'experts france begin examining airplane debris found reunion island french air accident experts wedn'

In [23]:
print(x_train_tokens[800])


[805, 895, 1141, 1334, 353, 156, 141, 402, 284, 1335, 224, 55, 805, 4263]


In [24]:
tokens_to_string(x_train_tokens[800])


'experts france begin examining airplane debris found reunion island french air accident experts wedn'

In [25]:
filtered_stopwords = []
filtered_stopwords_list = []

for i in data:
    filtered_sentence = [w for w in i]
    filtered_stopwords_list.append(filtered_sentence)                         #return list value
    filtered_stopwords.append(" ".join(filtered_sentence))                    #return string value

In [47]:
#Save word2vec format (not binary)

model = Word2Vec(filtered_stopwords_list, size=1000)
model_save_location = "3000tweets_notbinary"
model.wv.save_word2vec_format(model_save_location)

2020-08-06 18:40:41,615 : INFO : collecting all words and their counts
2020-08-06 18:40:41,616 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-06 18:40:41,669 : INFO : collected 28 word types from a corpus of 456453 raw words and 7613 sentences
2020-08-06 18:40:41,670 : INFO : Loading a fresh vocabulary
2020-08-06 18:40:41,670 : INFO : effective_min_count=5 retains 28 unique words (100% of original 28, drops 0)
2020-08-06 18:40:41,671 : INFO : effective_min_count=5 leaves 456453 word corpus (100% of original 456453, drops 0)
2020-08-06 18:40:41,672 : INFO : deleting the raw counts dictionary of 28 items
2020-08-06 18:40:41,672 : INFO : sample=0.001 downsamples 23 most-common words
2020-08-06 18:40:41,673 : INFO : downsampling leaves estimated 79036 word corpus (17.3% of prior 456453)
2020-08-06 18:40:41,674 : INFO : estimated required memory for 28 words and 1000 dimensions: 238000 bytes
2020-08-06 18:40:41,674 : INFO : resetting layer weights
2020-08

In [59]:
embedding_size = 1000


In [60]:

#Word2vec load(2.option) example

word2vec = {}
with open('3000tweets_notbinary', encoding='UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

In [61]:

print("x_test[0] :",x_test[40])
print("x_test_pad[0] :",x_test_pad[0])

x_test[0] : alex making watch facts minions want fall cliff help
x_test_pad[0] : [   0    0    0    0    0    0    0  685 2312   25   89  709 1060 4480
  671 4481]


In [62]:
num_words = len(list(tokenizer.word_index)) + 1


In [63]:
embedding_matrix = np.random.uniform(-1, 1, (num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    if i < num_words:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [64]:
embedding_matrix.shape


(14369, 1000)

In [65]:
embedding_matrix[3]


array([-5.93409523e-01,  3.84118369e-01,  6.95945573e-01,  7.29381563e-01,
        5.53385736e-01,  2.24626343e-01, -3.05124016e-01, -5.91050699e-01,
       -2.11955462e-01, -8.88838877e-01, -8.16972188e-01, -4.87341610e-01,
        9.66319900e-01,  3.15930875e-01,  4.70544899e-01,  2.76626520e-01,
       -6.28581320e-01,  1.58175821e-01,  2.34742139e-01, -2.26463842e-01,
       -6.74042861e-02,  4.95473198e-01,  7.35480654e-01, -1.53020114e-01,
       -6.27996585e-01,  8.34729513e-01,  1.61373133e-01, -9.38528590e-01,
        6.07812038e-01, -9.21655357e-01, -7.81224773e-01,  9.75251348e-02,
       -3.78557272e-01,  1.21502409e-01, -9.08726531e-01,  8.65587050e-01,
       -7.06353924e-01, -6.51708505e-01, -2.14730170e-01,  4.53398779e-02,
       -7.27353696e-01, -2.44400020e-01,  6.69351384e-01,  5.73899714e-01,
        2.19046228e-01, -6.85704461e-01,  7.06881574e-01,  1.95838391e-01,
       -9.37791741e-02, -4.87151030e-01,  4.57760351e-01,  3.65106162e-01,
        2.76482071e-01,  

In [66]:
sequence_length = max_tokens
vocabulary_size = num_words
embedding_dim = embedding_size
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

In [67]:
epochs = 5
batch_size = 30

In [68]:
y_train2 = np.array([])
y_test2 = np.array([])
for i in y_train:
    y_train2 = np.append(y_train2, i)
for i in y_test:
    y_test2 =np.append(y_test2, i)

In [71]:
#CNN architecture

num_classes = 2

#Training params
batch_size = 64 
num_epochs = 25

#Model parameters
num_filters = 16  # görüntünün boyutu mesela 512*512
embed_dim = embedding_size 
weight_decay = 1e-4

print("training CNN ...")
model = Sequential()

#Model add word2vec embedding

model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    weights= [embedding_matrix],
                    input_length=max_tokens,        
                    trainable=True,              #the layer is trained
                    name='embedding_layer'))
model.add(Conv1D(num_filters, 7, activation='tanh', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='tanh', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.9))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(num_classes, activation='softmax'))  #multi-label (k-hot encoding)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()



#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=8, verbose=1)
callbacks_list = [early_stopping]


hist = model.fit(x_train_pad, y_train2, batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=2)

training CNN ...
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 16, 1000)          14369000  
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 16, 16)            112016    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 8, 16)             0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 8, 16)             1808      
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 16)                0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 3

In [73]:
predicted = model.predict(x_test_pad)
predicted =np.argmax(predicted, axis=1) 
np.mean(predicted == y_test2)

0.7537754432042022